In [ ]:
# Add project src to path.
import set_path

import time
import numpy as np

from src.region_growing.label_connected_comp import LabelConnectedComp
from src.utils.las_utils import read_las
from src.utils.labels import Labels

In [ ]:
tilecode = '2386_9702'

in_file = '../datasets/pointcloud/processed_' + tilecode + '.laz'

In [ ]:
pointcloud = read_las(in_file)
points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
labels = pointcloud.label

In [ ]:
mask = labels == Labels.STREET_LIGHT

In [ ]:
lcc = LabelConnectedComp(Labels.STREET_LIGHT)
point_components = lcc.get_components(points[mask])

In [ ]:
min_component_size = 500
max_component_size = 10000

mask_indices = np.where(mask)[0]
    
cc_labels, counts = np.unique(point_components,
                              return_counts=True)

cc_labels_filtered = cc_labels[(counts >= min_component_size) & (counts <= max_component_size)]

for cc in cc_labels_filtered:
    # select points that belong to the cluster
    cc_mask = (point_components == cc)
    
    xyz = points[mask_indices[cc_mask]]
    x_points = xyz[:,0]
    y_points = xyz[:,1]
    
    # Calculate the centroid of the cluster
    centroid = (round(sum(x_points) / len(xyz),2), round(sum(y_points) / len(xyz),2))
    print(centroid)
    
    plt.scatter(x_points, y_points, alpha=0.5)
plt.show()